In [1]:
from pyspark.sql import SparkSession
import numpy as np
import copy

In [2]:
directory='file:///Users/khusaal/Documents/Masters/HPBDS/Project/netflix-prize-data/'
lines=sc.textFile(directory+'combined_data_1.txt')

In [13]:
rows=1000
def RDD_to_list(lines):
    RDDlist=[]
    for i in range(len(lines)):
        if ':' in lines[i]:
            movieID = lines[i].replace(':', '')
        else:
            temp=lines[i].split(',')
            temp.extend(temp[2].split('-'))
            del temp[2]
            temp.insert(0,int(movieID))
            RDDlist.append(temp)
    return RDDlist
RDD_data=sc.parallelize(RDD_to_list(lines.take(rows)))
RDD_data.take(5)

[[1, '1488844', '3', '2005', '09', '06'],
 [1, '822109', '5', '2005', '05', '13'],
 [1, '885013', '4', '2005', '10', '19'],
 [1, '30878', '4', '2005', '12', '26'],
 [1, '823519', '3', '2004', '05', '03']]

In [14]:
movies=RDD_data.map(lambda x:x[0]).distinct().count()
user=RDD_data.map(lambda x:int(x[1])).max()
print(movies,user)

3 2648861


In [15]:
RDD_user=RDD_data.map(lambda x:(x[1],x[0])).groupByKey().mapValues(list).map(lambda x:[int(x[0]),(x[1],1/len(x[1]))])
RDD_user.take(5)

[[822109, ([1], 1.0)],
 [885013, ([1], 1.0)],
 [823519, ([1], 1.0)],
 [893988, ([1], 1.0)],
 [2207774, ([1], 1.0)]]

In [16]:
start_date=int(RDD_data.map(lambda x:x[3]).min())
print(start_date)

2003


In [17]:
def calculateMonths(line):
    return [int(line[1]),(line[0],int(line[2]),(int(line[3])-start_date)*12+int(line[4])+int(line[5])/30.0)]
RDD_time=RDD_data.map(lambda x:calculateMonths(x))
RDD_time.take(5)

[[1488844, (1, 3, 33.2)],
 [822109, (1, 5, 29.433333333333334)],
 [885013, (1, 4, 34.63333333333333)],
 [30878, (1, 4, 36.86666666666667)],
 [823519, (1, 3, 17.1)]]

In [18]:
RDD_join=RDD_user.join(RDD_time)
RDD_join.take(5)

[(893988, (([1], 1.0), (1, 3, 35.56666666666667))),
 (1308744, (([1], 1.0), (1, 5, 34.96666666666667))),
 (4326, (([1], 1.0), (1, 4, 34.96666666666667))),
 (1196100, (([1], 1.0), (1, 4, 23.0))),
 (1834590, (([1], 1.0), (1, 3, 26.166666666666668)))]

In [19]:
def findLastMovie(x):
    diff=np.zeros(len(x[1]))
    movie=np.zeros(len(x[1]))
    output=[]
    for i in range(len(x[1])):
        movie[i]=x[1][i][1][0]
        diff[i]=x[1][i][1][2]
    for i in range(len(x[1])):
        d=copy.deepcopy(diff)
        d[i]=10*diff[i]
        last_movie=-1 if x[1][i][1][2]==np.min(diff) else movie[np.argmin(abs(d-x[1][i][1][2]))]
        output.append([x[1][i][1][1],x[0],movie[i],x[1][i][0],diff[i],last_movie])
    return output
RDD_final=RDD_join.groupByKey().mapValues(list).map(lambda x:findLastMovie(x)).collect()
RDD_final[:5]

[[[3, 893988, 1.0, ([1], 1.0), 35.56666666666667, -1]],
 [[5, 1308744, 1.0, ([1], 1.0), 34.96666666666667, -1]],
 [[4, 4326, 1.0, ([1], 1.0), 34.96666666666667, -1]],
 [[4, 1196100, 1.0, ([1], 1.0), 23.0, -1]],
 [[3, 1834590, 1.0, ([1], 1.0), 26.166666666666668, -1]]]

In [20]:
data=sc.parallelize([a for i in range(len(RDD_final)) for a in RDD_final[i]])
data.take(5)

[[3, 893988, 1.0, ([1], 1.0), 35.56666666666667, -1],
 [5, 1308744, 1.0, ([1], 1.0), 34.96666666666667, -1],
 [4, 4326, 1.0, ([1], 1.0), 34.96666666666667, -1],
 [4, 1196100, 1.0, ([1], 1.0), 23.0, -1],
 [3, 1834590, 1.0, ([1], 1.0), 26.166666666666668, -1]]

In [21]:
def toLibfm(data):
    libfm=[str(data[0]),str(data[1]-1)+':1',str(int(user-1+data[2]))+':1']
    for i in range(len(data[3][0])):
        libfm.append(str(user-1+movies+data[3][0][i])+':'+str(data[3][1]))
    libfm.append(str(user+2*movies)+':'+str(data[4]))
    if data[5]!=-1:
        libfm.append(str(int(user+2*movies+data[5]))+':1')
    return ' '.join(i for i in libfm)
file_data=data.map(lambda x:toLibfm(x))
print(file_data.take(5))

['3 893987:1 2648861:1 2648864:1.0 2648867:35.56666666666667', '5 1308743:1 2648861:1 2648864:1.0 2648867:34.96666666666667', '4 4325:1 2648861:1 2648864:1.0 2648867:34.96666666666667', '4 1196099:1 2648861:1 2648864:1.0 2648867:23.0', '3 1834589:1 2648861:1 2648864:1.0 2648867:26.166666666666668']


In [23]:
file_data.saveAsTextFile('file:///Users/khusaal/Documents/Masters/HPBDS/Project/output')